<a href="https://colab.research.google.com/github/mwithgod3952/Machine_Learning_Theory_jh/blob/master/Try_new_TIme_seriese_KalmanFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/strongio/torch-kalman.git#egg=torch_kalman

  Cloning https://github.com/strongio/torch-kalman.git to /tmp/pip-install-t9s45x5i/torch-kalman
  Running command git clone -q https://github.com/strongio/torch-kalman.git /tmp/pip-install-t9s45x5i/torch-kalman
     |████████████████████████████████| 184kB 5.1MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
  Created wheel for torch-kalman: filename=torch_kalman-0.2.0-cp37-none-any.whl size=49262 sha256=91f33455a6995d4866b6738622ad6471b18176515e1c822989e308f224fced64
  Stored in directory: /tmp/pip-ephem-wheel-cache-ugzfboq0/wheels/c3/d1/5c/107d5ce5f9c0b22fae87937666c02fab8513d936ff9f73f166
  Created wheel for filterpy: filename=filterpy-1.4.5-cp37-none-any.whl size=110476 sha256=048550ce135e8db1f522661d57eed35a1f6f7c68a7fccf271eaa5005438eccd9
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
Successfully built torch-kalman filterpy


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import glob
import pandas as pd
import numpy as np

In [ ]:
local_path = '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)'
filenames = glob.glob(local_path + "/*.csv")
filenames

['/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/PRSA_Data_Aotizhongxin_20130301-20170228.csv',
 '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/PRSA_Data_Changping_20130301-20170228.csv',
 '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/PRSA_Data_Dingling_20130301-20170228.csv',
 '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/PRSA_Data_Dongsi_20130301-20170228.csv',
 '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/PRSA_Data_Guanyuan_20130301-20170228.csv',
 '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/PRSA_Data_Gucheng_20130301-20170228.csv',
 '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/PRSA_Data_Huairou_20130301-20170228.csv',
 '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/PRSA_Data_Nongzhanguan_20130301-20170228.csv',
 '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/PRSA_Data_Shunyi_20130301-20170228.csv',
 '/content/drive/MyDrive/PRSA2017_Data_20130301-20170228 (1)/

In [ ]:
multiple_df_aq = [pd.read_csv(x) for x in filenames]
np.shape(multiple_df_aq)

(12, 35064, 18)

In [ ]:
n = 0

for df_aq in multiple_df_aq:
    ds = []
    for i in range(len(df_aq)):
        ds.append(pd.to_datetime(str(df_aq.iloc[i,:]['year']) +  "-" + str(df_aq.iloc[i,:]['month']) + "-" + str(df_aq.iloc[i,:]['day'])).strftime("%Y-%m-%d"))
    multiple_df_aq[n]["ds"] = ds
    multiple_df_aq[n] = multiple_df_aq[n].drop(['No', 'year', 'month',	'day',	'hour'], axis = 1)

    # '열' 재배치
    cols = multiple_df_aq[n].columns.tolist()
    cols.remove('ds')
    multiple_df_aq[n] = multiple_df_aq[n][sum([['ds'], a], [])]

    n = n + 1 

In [ ]:
T = multiple_df_aq[0].copy()
T.groupby('ds').mean()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
ds,,,,,,,,,,,
2013-03-01,7.125000,10.750000,11.708333,22.583333,429.166667,63.875000,1.391667,1026.875000,-18.745833,0.0,3.254167
2013-03-02,30.750000,42.083333,36.625000,66.666667,824.916667,29.750000,0.616667,1026.850000,-15.937500,0.0,1.479167
2013-03-03,76.916667,120.541667,61.291667,81.000000,1620.625000,19.125000,5.566667,1014.608333,-12.316667,0.0,1.658333
2013-03-04,22.708333,44.583333,22.869565,46.956522,617.391304,53.750000,9.962500,1017.650000,-11.683333,0.0,2.404167
2013-03-05,148.875000,183.791667,93.875000,132.833333,2357.958333,68.458333,6.291667,1010.900000,-7.525000,0.0,1.129167
...,...,...,...,...,...,...,...,...,...,...,...
2017-02-24,21.541667,32.625000,16.583333,58.875000,575.000000,50.791667,5.166667,1021.012500,-11.141667,0.0,1.595833
2017-02-25,11.208333,19.708333,6.750000,43.375000,420.833333,65.875000,7.245833,1018.812500,-10.754167,0.0,1.825000
2017-02-26,28.125000,40.708333,10.083333,65.375000,720.833333,48.625000,7.208333,1020.337500,-8.258333,0.0,1.566667


In [ ]:
pd.to_datetime(T['ds']) + pd.to_timedelta(7, unit='d')

0       2013-03-08
1       2013-03-08
2       2013-03-08
3       2013-03-08
4       2013-03-08
           ...    
35059   2017-03-07
35060   2017-03-07
35061   2017-03-07
35062   2017-03-07
35063   2017-03-07
Name: ds, Length: 35064, dtype: datetime64[ns]

In [ ]:
df['Date'] = pd.to_datetime(df['Date']) - pd.to_timedelta(7, unit='d')
df = df.groupby(['Name', pd.Grouper(key='Date', freq='W-MON')])['Quantity']
       .sum()
       .reset_index()
       .sort_values('Date')